<a href="https://colab.research.google.com/github/yujeonghyeop/Bitcoin-Anomaly-Transaction-Detection/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
def load_data(data_dir, start_ts, end_ts):
    classes_csv = 'elliptic_txs_classes.csv'
    edgelist_csv = 'elliptic_txs_edgelist.csv'
    features_csv = 'elliptic_txs_features.csv'

    classes = pd.read_csv(os.path.join(data_dir, classes_csv), index_col = 'txId') # labels for the transactions i.e. 'unknown', '1', '2'
    edgelist = pd.read_csv(os.path.join(data_dir, edgelist_csv), index_col = 'txId1') # directed edges between transactions
    features = pd.read_csv(os.path.join(data_dir, features_csv), header = None, index_col = 0) # features of the transactions
##########################################

    num_features = features.shape[1]
    num_tx = features.shape[0] #feature 크기
    total_tx = list(classes.index) #0부터 203769까지 index화
	# select only the transactions which are labelled
    labelled_classes = classes[classes['class'] != 'unknown']   #labeld있는 친구들만 남김
    labelled_tx = list(labelled_classes.index)  #labeled있는 놈들의 index만 추출[3,9,10,11,16...] , 46564개
	# to calculate a list of adjacency matrices for the different timesteps

    adj_mats = []
    features_labelled_ts = []
    classes_ts = []
    num_ts = 49 # number of timestamps from the paper

    for ts in range(start_ts, end_ts):
        features_ts = features[features[1] == ts+1] #feature의 timestamp가 ts+1번째인 놈들 추출, timestamp가 1인 친구들은 7878개...
        tx_ts = list(features_ts.index) #timestamp가 ts+1인 친구들의 index를 list화 

        labelled_tx_ts = [tx for tx in tx_ts if tx in set(labelled_tx)] #tx_ts에 있는 놈들 중에서 labelled_tx에 있는 놈이라면 labelled_tx_ts에 추가한다.
        #Timestamp가 1인 놈들은 총 7878개가 있었는데, 그 중에 label이 있는 놈들은 2147개 이다.

	    # adjacency matrix for all the transactions
	    # we will only fill in the transactions of this timestep which have labels and can be used for training
        adj_mat = pd.DataFrame(np.zeros((num_tx, num_tx)), index = total_tx, columns = total_tx)

        edgelist_labelled_ts = edgelist.loc[edgelist.index.intersection(labelled_tx_ts).unique()]   #edgelist에서 timestamp가 1인놈에서 label이 있는 친구의 인덱스의 행을 가져옴
        for i in range(edgelist_labelled_ts.shape[0]):  #위의 변수의 크기니까 timestamp가 1인 친구들에서 label이 달린 친구의 갯수, 2147개 임
            adj_mat.loc[edgelist_labelled_ts.index[i], edgelist_labelled_ts.iloc[i]['txId2']] = 1   
        #관계가 있다는 것을 나타내기 위해서 2147개의 index([3,9,10,11,16])를 행 edgelist['txID2']의 값을 열로 만들어서 adj_mat(index, value)==1을 추가한다 

        adj_mat_ts = adj_mat.loc[labelled_tx_ts, labelled_tx_ts]
        features_l_ts = features.loc[labelled_tx_ts]    #timestamp가 1인 놈들 중 label이 있는 index의 feature행을 저장해준다.

        adj_mats.append(adj_mat_ts)
        features_labelled_ts.append(features_l_ts)  #결론적으로 timestamp가 있는 놈들 중 label이 있는 놈들만 담김
        classes_ts.append(classes.loc[labelled_tx_ts])  #timestamp가 있는 놈들 중 label이 있는 놈들의 class를 저장함
    return adj_mats, features_labelled_ts, classes_ts   #timestamp가 있는 놈들중 label이 있는 놈들만 넘김


In [ ]:

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, auc, precision_score,roc_curve
from sklearn.metrics import plot_precision_recall_curve, average_precision_score, f1_score, fbeta_score,plot_roc_curve,roc_auc_score,precision_recall_curve
import matplotlib.pyplot as plt   
import seaborn as sns
import numpy as np
import time
import sys
import pandas as pd
import os
dir = "./"
dataSet = load_data(dir, 0, 34)

import numpy as np
import time
import sys





In [ ]:
class GraphConv(nn.Module):
    def __init__(self, in_features, out_features, activation  = 'relu', skip = False, skip_in_features = None):
        super(GraphConv, self).__init__()
        self.W = torch.nn.Parameter(torch.DoubleTensor(in_features, out_features))
        nn.init.xavier_uniform_(self.W)

        self.set_act = False
        if activation == 'relu':
            self.activation = nn.ReLU()
            self.set_act = True
        elif activation == 'softmax':
            self.activation = nn.Softmax(dim = 1)
            self.set_act = True
        else:
            self.set_act = False
            raise ValueError("activations supported are 'relu' and 'softmax'")

        self.skip = skip
        if self.skip:
            if skip_in_features == None:
                raise ValueError("pass input feature size of the skip connection")
            self.W_skip = torch.nn.Parameter(torch.DoubleTensor(skip_in_features, out_features))
            nn.init.xavier_uniform_(self.W)

    def forward(self, A, H_in, H_skip_in = None):
        # A must be an n x n matrix as it is an adjacency matrix
        # H is the input of the node embeddings, shape will n x in_features
        self.A = A
        self.H_in = H_in            # self-loop 추가로 해당 자기자신 노드도 latent feature vector에 영향을 주게 추가
        A_ = torch.add(self.A, torch.eye(self.A.shape[0]).double())
        D_ = torch.diag(A_.sum(1))
        # since D_ is a diagonal matrix,
        # its root will be the roots of the diagonal elements on the principle diagonal
        # since A is an adjacency matrix, we are only dealing with positive values
        # all roots will be real
        D_root_inv = torch.inverse(torch.sqrt(D_))
        A_norm = torch.mm(torch.mm(D_root_inv, A_), D_root_inv) # A행렬을 정규화 함으로써 feature vector 안정화
        #정규화는 D^(-1/2)*A*D^(-1/2) 로 진행. D는 A의 차수행렬
        # shape of A_norm will be n x n

        H_out = torch.mm(torch.mm(A_norm, H_in), self.W)
        # shape of H_out will be n x out_features

        if self.skip:
            H_skip_out = torch.mm(H_skip_in, self.W_skip)
            H_out = torch.add(H_out, H_skip_out)

        if self.set_act:
            H_out = self.activation(H_out)

        return H_out


In [ ]:
class GCN_layer(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, skip = False):
        super(GCN_layer, self).__init__()
        self.skip = skip

        self.gcl1 = GraphConv(in_features, hidden_features)

        if self.skip:
            self.gcl_skip = GraphConv(hidden_features, out_features, activation = 'softmax', skip = self.skip,
                                  skip_in_features = in_features)
        else:
            self.gcl2 = GraphConv(hidden_features, out_features, activation = 'softmax')

    def forward(self, A, X):
        out = self.gcl1(A, X)
        if self.skip:
            out = self.gcl_skip(A, out, X)
        else:
            out = self.gcl2(A, out)

        return out


In [ ]:
#!mkdir modelDir
num_features = 166
num_classes = 2
num_ts = 49
epochs = 15
lr = 0.001
max_train_ts = 34
train_ts = np.arange(max_train_ts)

adj_mats, features_labelled_ts, classes_ts = dataSet

# 0 - illicit, 1 - licit
labels_ts = []
for c in classes_ts:
    labels_ts.append(np.array(c['class'] == '2', dtype = np.long))

gcn = GCN_layer(num_features, 100, num_classes)
train_loss = nn.CrossEntropyLoss(weight = torch.DoubleTensor([0.7, 0.3]))
optimizer = torch.optim.Adam(gcn.parameters(), lr = lr)



/var/folders/gp/87l6kx4d3wxb6ttn4y3zkvgh0000gn/T/ipykernel_18633/1352923489.py:15: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  labels_ts.append(np.array(c['class'] == '2', dtype = np.long))


In [ ]:
# Training

for ts in train_ts:
    A = torch.tensor(adj_mats[ts].values)
    X = torch.tensor(features_labelled_ts[ts].values)
    L = torch.tensor(labels_ts[ts], dtype = torch.long)
    for ep in range(epochs):
        t_start = time.time()

        gcn.train()
        optimizer.zero_grad()
        out = gcn(A, X)

        loss = train_loss(out, L)
        train_pred = out.max(1)[1].type_as(L)
        acc = (train_pred.eq(L).double().sum())/L.shape[0]

        loss.backward()
        optimizer.step()

        sys.stdout.write("\r Epoch %d/%d Timestamp %d/%d training loss: %f training accuracy: %f Time: %s"
                         %(ep, epochs, ts, max_train_ts, loss, acc, time.time() - t_start)
                        )
torch.save(gcn.state_dict(), str(os.path.join("./modelDir", "gcn_weights.pth")))
######################################################################################

# Epoch 14/15 Timestamp 33/34 training loss: 0.376320 training accuracy: 0.966990 Time: 0.049683570861816406
from sklearn.metrics import f1_score, precision_score, recall_score
test_ts = np.arange(14)
adj_mats, features_labelled_ts, classes_ts = load_data(dir, 35, 49)

# 0 - illicit, 1 - licit
labels_ts = []
for c in classes_ts:
    labels_ts.append(np.array(c['class'] == '2', dtype = np.long))

gcn = GCN_layer(num_features, 100, num_classes)
gcn.load_state_dict(torch.load(os.path.join("./modelDir", "gcn_weights.pth")))



 Epoch 14/15 Timestamp 33/34 training loss: 0.374216 training accuracy: 0.970874 Time: 0.0208618640899658286

/var/folders/gp/87l6kx4d3wxb6ttn4y3zkvgh0000gn/T/ipykernel_18633/3617977430.py:35: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  labels_ts.append(np.array(c['class'] == '2', dtype = np.long))


<All keys matched successfully>

In [ ]:
# Testing
test_accs = []
test_precisions = []
test_recalls = []
test_f1s = []

for ts in test_ts:
    A = torch.tensor(adj_mats[ts].values)
    X = torch.tensor(features_labelled_ts[ts].values)
    L = torch.tensor(labels_ts[ts], dtype = torch.long)
    df = pd.DataFrame(labels_ts[ts])
    gcn.eval()
    test_out = gcn(A, X)

    test_pred = test_out.max(1)[1].type_as(L)
    t_acc = (test_pred.eq(L).double().sum())/L.shape[0]
    test_accs.append(t_acc.item())
    test_precisions.append(precision_score(L, test_pred))
    test_recalls.append(recall_score(L, test_pred))
    test_f1s.append(f1_score(L, test_pred))

acc = np.array(test_accs).mean()
prec = np.array(test_precisions).mean()
rec = np.array(test_recalls).mean()
f1 = np.array(test_f1s).mean()

print("GCN - averaged accuracy: {}, precision: {}, recall: {}, f1: {}".format(acc, prec, rec, f1))
#GCN - averaged accuracy: 0.9550850204745525, precision: 0.9662446287088422, recall: 0.986410574362438, f1: 0.9759155462246619

GCN - averaged accuracy: 0.9531416511369578, precision: 0.9664509605839543, recall: 0.9841924228147118, f1: 0.9749124175965571


In [ ]:
print(adj_mats)

[           200459281  199944769  198827545  200997190  200996856  200620377  \
200459281        0.0        0.0        0.0        0.0        0.0        0.0   
199944769        0.0        0.0        0.0        0.0        0.0        0.0   
198827545        0.0        0.0        0.0        0.0        0.0        0.0   
200997190        0.0        0.0        0.0        0.0        1.0        0.0   
200996856        0.0        0.0        0.0        0.0        0.0        0.0   
...              ...        ...        ...        ...        ...        ...   
59760533         0.0        0.0        0.0        0.0        0.0        0.0   
66403063         0.0        0.0        0.0        0.0        0.0        0.0   
136314571        0.0        0.0        0.0        0.0        0.0        0.0   
93761483         0.0        0.0        0.0        0.0        0.0        0.0   
202510866        0.0        0.0        0.0        0.0        0.0        0.0   

           200618680  199991289  4209720    200032